## Continuing Cleaning using SQL via Python

Connection

In [ ]:
import pyodbc

In [ ]:
DRIVER_NAME = 'ODBC Driver 17 for SQL Server'
SERVER_NAME = r'DESKTOP-L3GBMQ5\SQLEXPRESS'
DATABASE_NAME = 'Healthcare_Case_Study'

connection_string = (
    f"DRIVER={{{DRIVER_NAME}}};"
    f"SERVER={SERVER_NAME};"
    f"DATABASE={DATABASE_NAME};"
    f"Trusted_Connection=yes;"
)

conn = pyodbc.connect(connection_string)
cursor = conn.cursor()
print("Connected successfully!")

Connected successfully!


Handle NULL Satisfaction Scores by replacing with department averages

In [ ]:
cursor.execute("""
UPDATE H
SET Satisfaction_Score = A.AvgSatisfaction
FROM Hospital_Operations_Cleaned_1 H
JOIN (
    SELECT 
        Department
        ,AVG(Satisfaction_Score) AS AvgSatisfaction
    FROM 
        Hospital_Operations_Cleaned_1
    WHERE 
        Satisfaction_Score IS NOT NULL
    GROUP BY 
        Department
) A
ON 
    H.Department = A.Department
WHERE 
    H.Satisfaction_Score IS NULL;
""")

Replace NULL Wait_Time _Minutes with dapartment median

In [ ]:
cursor.execute("""
    ;WITH DeptMedian AS (
        SELECT
            Department
            ,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY Wait_Time_Minutes)
                OVER (PARTITION BY Department) AS MedianWait
        FROM 
            Hospital_Operations_Cleaned_1
    )
UPDATE H
SET
    Wait_Time_Minutes = D.MedianWait
FROM 
    Hospital_Operations_Cleaned_1 H
JOIN 
    DeptMedian D 
        ON H.Department = D.Department
WHERE 
    H.Wait_Time_Minutes IS NULL;
""")

Standardize Insurance Type and Department text 

In [ ]:
cursor.execute("""
    UPDATE 
        Hospital_Operations_Cleaned_1
    SET
        Insurance_Type = CASE
        WHEN Insurance_Type IN ('Privte', 'Private','Privat') THEN 'Private'
        WHEN Insurance_Type IN ('Self Pay', 'Self-Pay','Selfpay') THEN 'Self-Pay'
        WHEN Insurance_Type IN ('Public ', 'Publick','Publik') THEN 'Public'
        ELSE Insurance_Type
        END
        ,Department = UPPER(LEFT(Department,1)) + LOWER(SUBSTRING(Department,2,LEN(Department)))
    ;
""")

Handle invalid or negative stay lenghs

In [ ]:
cursor.execute("""
    DELETE FROM
        Hospital_Operations_Cleaned_1
    WHERE
        Length_of_Stay < 0 OR Length_of_Stay IS NULL
        ;       
               
    """)

In [ ]:
cursor.execute("""
    UPDATE
        Hospital_Operations_Cleaned_1
    SET
        Readmission_30Days = CASE
            WHEN LOWER(Readmission_30Days) IN ('yes', 'y') THEN 'Y'
            WHEN LOWER(Readmission_30Days) IN ('no', 'n') THEN 'N'
            ELSE '0'END
        ,Gender = CASE
        WHEN LOWER(Gender) LIKE 'm%' THEN 'M'
        WHEN LOWER(Gender) LIKE 'f%' THEN 'F'
        ELSE 'U'END;               
""")

Commit all changes 

In [ ]:
conn.commit()

In [ ]:
import pandas as pd

In [ ]:
check_query = """
SELECT
    COUNT(*) AS TotalRecords
    ,COUNT(CASE WHEN Satisfaction_Score IS NULL THEN 1 END) AS Missing_Satisfaction
    ,COUNT(CASE WHEN Wait_Time_Minutes IS NULL THEN 1 END) AS Missing_Wait
    ,COUNT(CASE WHEN Length_of_Stay < 0 OR Length_of_Stay IS NULL THEN 1 END) AS Negative_Stay
    ,COUNT(DISTINCT Insurance_Type) AS Unique_Insurance_Types
FROM
    Hospital_Operations_Cleaned_1;
"""

In [ ]:
df_check = pd.read_sql(check_query, conn)
df_check

C:\Users\GIGABYTE\AppData\Local\Temp\ipykernel_10784\359547653.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_check = pd.read_sql(check_query, conn)


,TotalRecords,Missing_Satisfaction,Missing_Wait,Negative_Stay,Unique_Insurance_Types
0,1252,0,0,0,3


Save the Cleaned Table

In [ ]:
cursor.execute("""
    SELECT 
        *
    INTO 
        Hospital_Operations_Cleaned_2
    FROM
        Hospital_Operations_Cleaned_1
""")

In [ ]:
conn.commit()